In [1]:
import pandas as pd
import numpy as np
import joblib
import sys, os
from sklearn.preprocessing import KBinsDiscretizer, Normalizer

import matplotlib.pyplot as plt
import seaborn as sns

### Get all drawdowns

In [2]:
drawdowns = {
}

In [3]:
for file in os.listdir("D:/AlgoMLData/Drawdowns/"):
    ticker = file.split('_')[0]
    with open("D:/AlgoMLData/Drawdowns/" + file, 'rb') as file:
        drawdowns[ticker] = joblib.load(file)

In [4]:
all_drawdowns = {
    i : [] for i in range(100)
}

In [5]:
for ticker in drawdowns:
    for candle in drawdowns[ticker]:
        all_drawdowns[candle].extend(drawdowns[ticker][candle])

### Priors

In [6]:
bins = []
encoders = {}
for candle in all_drawdowns:
    kbd = KBinsDiscretizer(n_bins=256, encode='ordinal', strategy='uniform')
    kbd = kbd.fit(np.reshape(all_drawdowns[candle], (-1, 1)))
    encoders[candle] = kbd
    x = kbd.transform(np.reshape(all_drawdowns[candle], (-1, 1))).reshape(-1, )
    _, counts = np.unique(x, return_counts = True)
    bins.append(counts.tolist())
df = pd.DataFrame(bins)
df.fillna(0, inplace=True)
df = df.T

In [7]:
df / df.sum(axis=0)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.129686,0.102961,0.087730,0.084617,0.079920,0.069194,0.064497,0.057295,0.056475,0.051435,...,0.030707,0.030006,0.027232,0.027868,0.028329,0.028586,0.029039,0.028325,0.026300,0.025411
1,0.013807,0.029279,0.015928,0.033892,0.031791,0.029754,0.027461,0.029909,0.027113,0.025791,...,0.052167,0.051052,0.049878,0.050767,0.048906,0.051192,0.049498,0.048905,0.048291,0.046637
2,0.021281,0.039295,0.022353,0.048684,0.044913,0.042894,0.039673,0.040689,0.037376,0.035208,...,0.062262,0.062713,0.063279,0.061280,0.061875,0.058849,0.059617,0.060928,0.058544,0.059417
3,0.025912,0.049097,0.029563,0.058276,0.052861,0.049226,0.046084,0.050889,0.047312,0.045190,...,0.074968,0.070748,0.067938,0.071938,0.067671,0.068621,0.068710,0.067493,0.069762,0.067937
4,0.031698,0.050292,0.032359,0.061348,0.059201,0.054883,0.052264,0.052563,0.051835,0.046984,...,0.074968,0.076650,0.076967,0.073594,0.074772,0.074163,0.074577,0.071255,0.068648,0.071450
5,0.033945,0.058274,0.035736,0.062069,0.058314,0.055518,0.051948,0.057452,0.054623,0.053131,...,0.074615,0.075299,0.077612,0.073090,0.072019,0.075111,0.072890,0.072951,0.074368,0.072870
6,0.033660,0.055286,0.038297,0.060765,0.057463,0.055994,0.052770,0.054706,0.052374,0.051287,...,0.073274,0.067833,0.065788,0.070282,0.071946,0.070517,0.070543,0.065796,0.066568,0.066891
7,0.038763,0.057790,0.041784,0.058860,0.055671,0.053831,0.051589,0.055376,0.051694,0.053967,...,0.064874,0.064562,0.065358,0.064233,0.062600,0.064027,0.063357,0.062329,0.063447,0.062033
8,0.039756,0.050221,0.039208,0.054484,0.052750,0.051330,0.051294,0.050153,0.049937,0.046837,...,0.058732,0.060225,0.062993,0.061352,0.061585,0.060527,0.058591,0.061592,0.059064,0.056652
9,0.040600,0.052384,0.039632,0.048409,0.048924,0.048730,0.047919,0.049483,0.049070,0.046665,...,0.054073,0.056314,0.053605,0.054007,0.054267,0.051265,0.054997,0.053847,0.053418,0.052990


### Build problem statement

In [8]:
with open('D:/AlgoMLData/Progressions/EURCHF_progs.pkl', 'rb') as file:
    progs = joblib.load(file)

In [11]:
progs[2]

,index,Datetime,Trade_ID,Direction,CandleSize,Anchor,Open,High,Low,Close,RDD,MDD
100 2,100,2009-05-01 08:20:00,2,1.0,0.00146,1.51061,1.510610,1.511880,1.508960,1.509150,146.0,165.0
101 2,101,2009-05-01 08:25:00,2,1.0,0.00146,1.51061,1.509245,1.509740,1.509245,1.509550,106.0,136.5
102 2,102,2009-05-01 08:30:00,2,1.0,0.00146,1.51061,1.509480,1.509480,1.508750,1.508915,169.5,186.0
103 2,103,2009-05-01 08:35:00,2,1.0,0.00146,1.51061,1.508925,1.509210,1.508680,1.508955,165.5,193.0
104 2,104,2009-05-01 08:40:00,2,1.0,0.00146,1.51061,1.508965,1.509610,1.508720,1.509455,115.5,189.0
105 2,105,2009-05-01 08:45:00,2,1.0,0.00146,1.51061,1.509475,1.509475,1.509055,1.509070,154.0,155.5
106 2,106,2009-05-01 08:50:00,2,1.0,0.00146,1.51061,1.509095,1.509100,1.508680,1.508870,174.0,193.0
107 2,107,2009-05-01 08:55:00,2,1.0,0.00146,1.51061,1.508875,1.508875,1.508120,1.508370,224.0,249.0
108 2,108,2009-05-01 09:00:00,2,1.0,0.00146,1.51061,1.508380,1.508740,1.507370,1.507445,316.5,324.0
109 2,109,2009-05-01 09:05:00,2,1.0,0.00146,1.51061,1.507435,1.508890,1.507435,1.508775,183.5,317.5
